In [ ]:
import logging

import numpy as np
from jaxmod.constants import EARTH_MASS

from atmodeller import EquilibriumModel, Planet, SpeciesNetwork, debug_logger

logger = debug_logger()
logger.setLevel(logging.INFO)

# For more output use DEBUG
# logger.setLevel(logging.DEBUG)

# Atmosphere

This notebook is available at `notebooks/atmosphere.ipynb` and is easiest to obtain by downloading the source code.

*Atmodeller* allows you to simultaneuosly solve for surface conditions and an atmospheric P-T profile under equilibrium assumptions. In this example, we follow the model presented in Hakim et al. (2025) for TOI-421b.

In [ ]:
earth_radius = 6371000  # metre

# Mass and radius of TOI-421b
planet_mass = 6.7 * EARTH_MASS  # kg
MEB_radius = 1.65 * earth_radius  # metre
# atm_radius = 2.64 * earth_radius  # metre
# MEB radius = 1.65 Earth radii in metre for 6.7 Earth masses (for atm_radius = 2.64 Earth radii)
# M-R relation from Hakim et al. (2018) Icarus

# Temperature of TOI-421b
surface_temperature = 3000  # K
top_temperature = 920  # K

We set up arrays for pressure (in bar) and temperature (in K) profile. The "trick" is to use ``np.nan`` as a placeholder indicating that the surface pressure should be solved for based on the total volatile mass in the atmosphere, that is, by enforcing mechanical pressure balance at the surface. For all other entires where a pressure value is provided, that value is used directly. The temperature array aligns positionally with the pressure array: the first entry gives the surface temperature, and the last entry corresponds to the temperature at the final pressure level.

In [ ]:
temperature = np.array([surface_temperature, 1500, 1000, top_temperature])
pressure = np.array([np.nan, 1, 1e-2, 1e-4])

We then create the model using the gaseous and condensed species we wish to include:

In [ ]:
species = SpeciesNetwork.create(
    (
        # Gas species
        "H2O_g",
        "H2_g",
        "O2_g",
        "OSi_g",
        "H4Si_g",
        "CO2_g",
        "CO_g",
        "CH4_g",
        "N2_g",
        "NH3_g",
        "He_g",
        # Condensates
        "O2Si_bqz",
        "O2Si_aqz",
        "O2Si_bcrt",
        "C_cr",
        "CSi_b",
        "Si_cr",
        "N4Si3_cr",
    )
)

model = EquilibriumModel(species)

We set the mass constraints based on a previous *Atmodeller* calculation that accounted for real-gas behaviour and dissolution into a magma ocean. From that calculation, we extract elemental abundances in the gas phase only (i.e. the atmosphere) and use them as abundance constraints. The values below correspond to a model with solar metallicity and a fully molten mantle.

In [ ]:
mass_constraints = {
    "H": 5.73802837470845e22,
    "He": 7.13997e21,
    "C": 1.18962697880417e21,
    "N": 1.99427e17,
    "O": 1.94361960955633e22,
    "Si": 3.5648e23,
}

Since we are using the gas masses from the previous calculation directly (which already accounted for gas solubility), we set ``mantle_melt_fraction=0``. The specification of the planet's mass and surface radius is used only when computing the mechanical pressure balance&mdash;that is, for determining the first pressure entry. These parameters play no role when the pressure profile is prescribed directly.

In [ ]:
planet = Planet(
    temperature=temperature,
    planet_mass=planet_mass,
    mantle_melt_fraction=0,
    surface_radius=MEB_radius,
    pressure=pressure,
)

Finally, we solve the model:

In [ ]:
model.solve(state=planet, mass_constraints=mass_constraints)

The output will contain four model evaluations, and you can see that the gas pressure satisfies the prescribed constraints, with the first entry enforcing the mechanical pressure balance at the surface. This workflow therefore provides a convenient way to evaluate the atmspheric chemical signature at different temperatures and pressures while remaining consistent with the planetary surface conditions.

In [ ]:
# Show the system
model.output.to_dataframes()["state"]

If you're interested in what happens "behind the scenes," this works because the gas volume is not required during the solution itself; instead, it is computed deterministically during post-processing. This allows the total number of moles to remain fixed&mdash;as in the example above&mdash;while still accommodating the specification of different pressures, since the gas volume can adjust to maintain thermodynamic consistency with the imposed conditions.